In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is NOT available")

GPU is NOT available


In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from evaluator_ANAELE import *

'''import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, SparseCategoricalAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau'''



ModuleNotFoundError: No module named 'evaluator_ANAELE'

In [16]:
# Load pickle file and convert to numpy array
with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)
 
#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

X_test_true = dat['X_test']
S_test_true = dat['S_test']

print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)

# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')

# Split the data (final _ to keep split data untouched and be able to reload in file)
X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_ = train_test_split(X, Y, S, test_size=0.3, random_state=42)

# Refresh training data
X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_

dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


In [17]:

def classwise_loss(y_true, y_pred):
    """
    Custom loss function that calculates the cross-entropy loss for each class
    individually and then averages these losses.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted labels, as probabilities.
    
    Returns:
    - loss: A scalar tensor representing the average class-wise cross-entropy loss.
    """
        # Ensure y_true and y_pred are both float32 for compatibility
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    # Ensure predictions sum to 1
    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    y_pred = tf.math.divide(y_pred, tf.reduce_sum(y_pred, axis=-1, keepdims=True))
    
    # Calculate cross-entropy loss for each class
    classwise_losses = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=0)
    
    # Average the class-wise losses
    loss = tf.reduce_mean(classwise_losses)
    return loss


In [31]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class GradientReversalLayer(Layer):
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha

    def call(self, x):
        return x

    def compute_output_shape(self, input_shape):
        return input_shape

    @tf.custom_gradient
    def grad_reverse(x, grad):
        return x, -grad * self.alpha

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'alpha': self.alpha
        })
        return config


In [36]:
# Define model architecture
input_layer = Input(shape=(768,))
shared_layer = Dense(256, activation='relu', name='shared_representation')(input_layer)

# Main task classifier
#main_classifier = Dense(128, activation='relu')(shared_layer)
main_classifier = Dropout(0.5)(shared_layer)#main_classifier)
main_output = Dense(28, activation='softmax', name='main_output')(main_classifier)

# Apply Gradient Reversal Layer before the adversary classifier
grl = GradientReversalLayer(alpha=0.1)(shared_layer)

# Adversary classifier (Adjusted for binary classification)
adversary_classifier = Dense(128, activation='relu')(grl)
adversary_classifier = Dropout(0.5)(adversary_classifier)
adversary_output = Dense(1, activation='sigmoid', name='adversary_output')(adversary_classifier)  # Adjusted

# Build model
model = Model(inputs=input_layer, outputs=[main_output, adversary_output])

# Compile model with adjusted loss and metrics
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss={'main_output': 'sparse_categorical_crossentropy', 'adversary_output': 'binary_crossentropy'},  # Adjusted classwise_loss / 'sparse_categorical_crossentropy'
              loss_weights={'main_output': 1., 'adversary_output': -0.0001},  # Negative weight to adversary for debiasing
              metrics={'main_output': SparseCategoricalAccuracy(name='accuracy'), 'adversary_output': AUC(name='auc')})

# Callback to reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=1)

# Train model
history = model.fit(X_train, {'main_output': Y_train, 'adversary_output': S_train},
                    epochs=10, batch_size=64, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/10
243/243 [==============================] - 3s 8ms/step - loss: 2.2830 - main_output_loss: 2.2833 - adversary_output_loss: 3.1513 - main_output_accuracy: 0.4115 - adversary_output_auc: 0.4724 - val_loss: 1.6796 - val_main_output_loss: 1.6804 - val_adversary_output_loss: 7.5507 - val_main_output_accuracy: 0.5591 - val_adversary_output_auc: 0.5000 - lr: 1.0000e-04
Epoch 2/10
243/243 [==============================] - 2s 7ms/step - loss: 1.6250 - main_output_loss: 1.6265 - adversary_output_loss: 15.6673 - main_output_accuracy: 0.5599 - adversary_output_auc: 0.5000 - val_loss: 1.3585 - val_main_output_loss: 1.3612 - val_adversary_output_loss: 26.4842 - val_main_output_accuracy: 0.6147 - val_adversary_output_auc: 0.5000 - lr: 1.0000e-04
Epoch 3/10
243/243 [==============================] - 1s 6ms/step - loss: 1.3886 - main_output_loss: 1.3927 - adversary_output_loss: 40.6770 - main_output_accuracy: 0.6095 - adversary_output_auc: 0.5000 - val_loss: 1.1891 - val_main_output_loss: 1.

In [33]:
# Evaluate model on test set
evaluation = model.evaluate(X_test, {'main_output': Y_test, 'adversary_output': S_test})


261/261 [==============================] - 1s 3ms/step - loss: 0.0360 - main_output_loss: 0.8787 - adversary_output_loss: 1685.4054 - main_output_accuracy: 0.7356 - adversary_output_auc: 0.5000


In [34]:
# Extract shared representation model
representation_model = Model(inputs=model.input, outputs=model.get_layer('shared_representation').output)

# Transform X_train and X_test
X_train_transformed_2 = representation_model.predict(X_train)
X_test_transformed_2 = representation_model.predict(X_test)

261/261 [==============================] - 1s 2ms/step


In [35]:
# Step 2: Train a new classifier on the transformed training data
clf_2 = LogisticRegression(max_iter=5000)  # Increase max_iter if needed for convergence
history_new_2 = clf_2.fit(X_train_transformed_2, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_2 = clf_2.predict(X_test_transformed_2)
accuracy_2= accuracy_score(Y_test, Y_pred_2)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_2}")

# Step 4 : Predict with gloabl score
eval_scores_2, confusion_matrices_eval_2 = gap_eval_scores(Y_pred_2, Y_test, S_test, metrics=['TPR'])
final_score_2 = (eval_scores_2['macro_fscore']+ (1-eval_scores_2['TPR_GAP']))/2
print('\nfinal',final_score_2)
print('macro_fscore',eval_scores_2['macro_fscore'])
print('1-eval_scores',1-eval_scores_2['TPR_GAP'])

Accuracy on transformed test data: 0.7577177177177177

final 0.7319076217263948
macro_fscore 0.6526436809324047
1-eval_scores 0.8111715625203849


In [39]:
classwise_loss(Y_test, Y_pred_2)
classwise_accuracy(Y_test, Y_pred_2)

2024-03-06 22:06:17.056611: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at reduction_ops_common.h:147 : INVALID_ARGUMENT: Invalid reduction dimension (0 for input with 0 dimension(s)


InvalidArgumentError: {{function_node __wrapped__Mean_device_/job:localhost/replica:0/task:0/device:CPU:0}} Invalid reduction dimension (0 for input with 0 dimension(s) [Op:Mean] name: 

In [38]:
import tensorflow as tf

def classwise_accuracy(y_true, y_pred):
    """
    Custom metric function that calculates the accuracy for each class individually
    and then averages these accuracies.
    
    Args:
        y_true: Tensor of true labels, one-hot encoded.
        y_pred: Tensor of predicted labels, as probabilities.

    Returns:
        A scalar tensor representing the average class-wise accuracy.
    """
    # Convert probabilities to predicted class (highest probability)
    y_pred_classes = tf.argmax(y_pred, axis=-1)
    y_true_classes = tf.argmax(y_true, axis=-1)
    
    # Calculate accuracy for each class
    class_accuracies = tf.cast(tf.equal(y_true_classes, y_pred_classes), tf.float32)
    
    # Reduce across all dimensions but the first (batch dimension)
    # to get accuracy per class, then take the mean across classes
    classwise_acc = tf.reduce_mean(class_accuracies, axis=0)
    overall_acc = tf.reduce_mean(classwise_acc)
    
    return overall_acc

# Example usage with a model compilation in TensorFlow/Keras
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[classwise_accuracy])
